In [1]:
pip install mamkit


  Using cached transformers-4.48.3-py3-none-any.whl.metadata (44 kB)
Using cached transformers-4.48.3-py3-none-any.whl (9.7 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.50.2
    Uninstalling transformers-4.50.2:
      Successfully uninstalled transformers-4.50.2


In [1]:
from mamkit.data.datasets import MMUSEDFallacy, InputMode
from pathlib import Path

def loading_data_example():
    base_data_path = Path('Dataset-textonly-testdata')
    loader = MMUSEDFallacy(task_name='afc',
                           input_mode=InputMode.TEXT_ONLY,  # or TEXT_AUDIO or AUDIO_ONLY
                           base_data_path=base_data_path)
    split_info = loader.get_splits('mm-argfallacy-2025')

In [2]:
from pathlib import Path
from mamkit.data.datasets import MMUSEDFallacy, InputMode

def loading_data_example():
    base_data_path = Path(__file__).parent.parent.resolve().joinpath('data')
    loader = MMUSEDFallacy(
        task_name='afd',
        input_mode=InputMode.TEXT_ONLY,  # Options: TEXT_ONLY, AUDIO_ONLY, TEXT_AUDIO
        base_data_path=base_data_path
    )
    split_info = loader.get_splits('mm-argfallacy-2025')


In [3]:

loader.split_info()

# Iterate over the splits to find the test split
for split_info in splits:
    if split_info.split_name == 'test':
        test_data = split_info.dataset
        break
else:
    raise ValueError("Test split not found.")

NameError: name 'loader' is not defined

========================


In [11]:
from pathlib import Path
from mamkit.data.datasets import MMUSEDFallacy, InputMode

def load_test_split():
    # Set base path to your dataset folder
    base_data_path = Path.cwd().parent.parent.resolve().joinpath('data')

    # Initialize the dataset loader
    loader = MMUSEDFallacy(
        task_name='afc',  # or 'afc' depending on your setup
        input_mode=InputMode.TEXT_ONLY,  # Can also be TEXT_AUDIO or AUDIO_ONLY
        base_data_path=base_data_path
    )

    # Get the train/val/test splits
    splits = loader.get_splits('mm-argfallacy-2025')

    # Find and return the test split
    for split_info in splits:
        if split_info.split_name == 'test':
            return split_info.dataset

    raise ValueError("Test split not found.")

# Example usage
test_data = load_test_split()
print(f"Test dataset contains {len(test_data.inputs)} samples.")


Downloading: https://zenodo.org/api/records/14938117/files-archive: 5.71MB [00:07, 836kB/s] 
  0%|          | 0/35 [00:00<?, ?it/s]

[youtube] Extracting URL: https://www.youtube.com/watch?v=_hEeMvhcNlE&t=4579s
[youtube] _hEeMvhcNlE: Downloading webpage
[youtube] _hEeMvhcNlE: Downloading tv client config
[youtube] _hEeMvhcNlE: Downloading player 9a279502
[youtube] _hEeMvhcNlE: Downloading tv player API JSON
[youtube] _hEeMvhcNlE: Downloading ios player API JSON
[youtube] _hEeMvhcNlE: Downloading m3u8 information
[info] _hEeMvhcNlE: Downloading 1 format(s): 251
[download] Destination: C:\Users\waral\Desktop\data\MMUSED-fallacy\audio_recordings\30_2004\full_audio
[download] 100% of   79.80MiB in 00:00:06 at 12.66MiB/s    
[ExtractAudio] Destination: C:\Users\waral\Desktop\data\MMUSED-fallacy\audio_recordings\30_2004\full_audio.wav
Deleting original file C:\Users\waral\Desktop\data\MMUSED-fallacy\audio_recordings\30_2004\full_audio (pass -k to keep)


  3%|▎         | 1/35 [00:21<12:27, 21.97s/it]

[youtube] Extracting URL: https://www.youtube.com/watch?v=xdQK78l3Xhc&t=119s
[youtube] xdQK78l3Xhc: Downloading webpage
[youtube] xdQK78l3Xhc: Downloading tv client config
[youtube] xdQK78l3Xhc: Downloading player 9a279502
[youtube] xdQK78l3Xhc: Downloading tv player API JSON
[youtube] xdQK78l3Xhc: Downloading ios player API JSON
[youtube] xdQK78l3Xhc: Downloading m3u8 information
[info] xdQK78l3Xhc: Downloading 1 format(s): 251
[download] Destination: C:\Users\waral\Desktop\data\MMUSED-fallacy\audio_recordings\5_1976\full_audio
[download] 100% of   82.12MiB in 00:00:06 at 12.14MiB/s    
[ExtractAudio] Destination: C:\Users\waral\Desktop\data\MMUSED-fallacy\audio_recordings\5_1976\full_audio.wav
Deleting original file C:\Users\waral\Desktop\data\MMUSED-fallacy\audio_recordings\5_1976\full_audio (pass -k to keep)


  6%|▌         | 2/35 [00:44<12:20, 22.45s/it]

[youtube] Extracting URL: https://www.youtube.com/watch?v=HZCcSTz1qLo
[youtube] HZCcSTz1qLo: Downloading webpage
[youtube] HZCcSTz1qLo: Downloading tv client config
[youtube] HZCcSTz1qLo: Downloading player 9a279502
[youtube] HZCcSTz1qLo: Downloading tv player API JSON
[youtube] HZCcSTz1qLo: Downloading ios player API JSON
[youtube] HZCcSTz1qLo: Downloading m3u8 information
[info] HZCcSTz1qLo: Downloading 1 format(s): 251
[download] Destination: C:\Users\waral\Desktop\data\MMUSED-fallacy\audio_recordings\23_1996\full_audio
[download] 100% of   63.21MiB in 00:00:05 at 12.07MiB/s    
[ExtractAudio] Destination: C:\Users\waral\Desktop\data\MMUSED-fallacy\audio_recordings\23_1996\full_audio.wav
Deleting original file C:\Users\waral\Desktop\data\MMUSED-fallacy\audio_recordings\23_1996\full_audio (pass -k to keep)


  9%|▊         | 3/35 [01:03<11:04, 20.75s/it]

[youtube] Extracting URL: https://www.youtube.com/watch?v=fj5A38gKHfk
[youtube] fj5A38gKHfk: Downloading webpage
[youtube] fj5A38gKHfk: Downloading tv client config
[youtube] fj5A38gKHfk: Downloading player 9a279502
[youtube] fj5A38gKHfk: Downloading tv player API JSON
[youtube] fj5A38gKHfk: Downloading ios player API JSON
[youtube] fj5A38gKHfk: Downloading m3u8 information
[info] fj5A38gKHfk: Downloading 1 format(s): 251
[download] Destination: C:\Users\waral\Desktop\data\MMUSED-fallacy\audio_recordings\6_1976\full_audio
[download] 100% of   58.22MiB in 00:00:04 at 12.29MiB/s    
[ExtractAudio] Destination: C:\Users\waral\Desktop\data\MMUSED-fallacy\audio_recordings\6_1976\full_audio.wav


  9%|▊         | 3/35 [01:22<14:43, 27.61s/it]


KeyboardInterrupt: 

=====================

In [15]:
from mamkit.data.datasets import MMUSEDFallacy, InputMode

# Specify the path to the dataset
base_data_path = 'path_to_downloaded_dataset'

# Initialize the dataset loader
loader = MMUSEDFallacy(
    task_name='afd',
    input_mode=InputMode.TEXT_ONLY,
    base_data_path=base_data_path
)

# Retrieve the data splits
split_info = loader.get_splits('mm-argfallacy-2025')

# Access the test dataset
test_dataset = split_info.test


Downloading: https://zenodo.org/api/records/14938117/files-archive: 5.71MB [00:07, 823kB/s] 
  0%|          | 0/35 [00:00<?, ?it/s]

[youtube] Extracting URL: https://www.youtube.com/watch?v=_hEeMvhcNlE&t=4579s
[youtube] _hEeMvhcNlE: Downloading webpage
[youtube] _hEeMvhcNlE: Downloading tv client config
[youtube] _hEeMvhcNlE: Downloading player 9599b765
[youtube] _hEeMvhcNlE: Downloading tv player API JSON
[youtube] _hEeMvhcNlE: Downloading ios player API JSON


         Install PhantomJS to workaround the issue. Please download it from https://phantomjs.org/download.html
         n = z_HxBAR-TDxqFnE ; player = https://www.youtube.com/s/player/9599b765/player_ias.vflset/en_US/base.js
         Install PhantomJS to workaround the issue. Please download it from https://phantomjs.org/download.html
         n = 4DMikeZjeS9dH5y ; player = https://www.youtube.com/s/player/9599b765/player_ias.vflset/en_US/base.js
         Install PhantomJS to workaround the issue. Please download it from https://phantomjs.org/download.html
         n = stXm1Vbmf08tLxS ; player = https://www.youtube.com/s/player/9599b765/player_ias.vflset/en_US/base.js
         Install PhantomJS to workaround the issue. Please download it from https://phantomjs.org/download.html
         n = Z8sT9-uKITNKpu6 ; player = https://www.youtube.com/s/player/9599b765/player_ias.vflset/en_US/base.js


[youtube] _hEeMvhcNlE: Downloading m3u8 information
[info] _hEeMvhcNlE: Downloading 1 format(s): 234
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 1094
[download] Destination: C:\Users\waral\Desktop\fyp\MM-ArgFallacy2025\path_to_downloaded_dataset\MMUSED-fallacy\audio_recordings\30_2004\full_audio
[download] 100% of   90.82MiB in 00:01:05 at 1.39MiB/s                     
[ExtractAudio] Destination: C:\Users\waral\Desktop\fyp\MM-ArgFallacy2025\path_to_downloaded_dataset\MMUSED-fallacy\audio_recordings\30_2004\full_audio.wav
Deleting original file C:\Users\waral\Desktop\fyp\MM-ArgFallacy2025\path_to_downloaded_dataset\MMUSED-fallacy\audio_recordings\30_2004\full_audio (pass -k to keep)


  3%|▎         | 1/35 [01:16<43:14, 76.30s/it]

...

[youtube] Extracting URL: https://www.youtube.com/watch?v=UCA1A5GqCdQ&ab_channel=NBCNews
[youtube] UCA1A5GqCdQ: Downloading webpage
[youtube] UCA1A5GqCdQ: Downloading tv client config
[youtube] UCA1A5GqCdQ: Downloading player 9599b765
[youtube] UCA1A5GqCdQ: Downloading tv player API JSON
[youtube] UCA1A5GqCdQ: Downloading ios player API JSON


         Install PhantomJS to workaround the issue. Please download it from https://phantomjs.org/download.html
         n = m5IgQ5CqpGyH0K9 ; player = https://www.youtube.com/s/player/9599b765/player_ias.vflset/en_US/base.js
         Install PhantomJS to workaround the issue. Please download it from https://phantomjs.org/download.html
         n = 1TevfuXMIL9rKg- ; player = https://www.youtube.com/s/player/9599b765/player_ias.vflset/en_US/base.js
         Install PhantomJS to workaround the issue. Please download it from https://phantomjs.org/download.html
         n = LqmJPFHG21VtuNs ; player = https://www.youtube.com/s/player/9599b765/player_ias.vflset/en_US/base.js
         Install PhantomJS to workaround the issue. Please download it from https://phantomjs.org/download.html
         n = KnmI7x0ey6g6RaP ; player = https://www.youtube.com/s/player/9599b765/player_ias.vflset/en_US/base.js


[youtube] UCA1A5GqCdQ: Downloading m3u8 information
[info] UCA1A5GqCdQ: Downloading 1 format(s): 234
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 1345
[download] Destination: C:\Users\waral\Desktop\fyp\MM-ArgFallacy2025\path_to_downloaded_dataset\MMUSED-fallacy\audio_recordings\46_2020\full_audio
[download] 100% of  111.60MiB in 00:01:20 at 1.38MiB/s                     
[ExtractAudio] Destination: C:\Users\waral\Desktop\fyp\MM-ArgFallacy2025\path_to_downloaded_dataset\MMUSED-fallacy\audio_recordings\46_2020\full_audio.wav
Deleting original file C:\Users\waral\Desktop\fyp\MM-ArgFallacy2025\path_to_downloaded_dataset\MMUSED-fallacy\audio_recordings\46_2020\full_audio (pass -k to keep)


Building clips...: 100%|██████████| 35/35 [04:05<00:00,  7.00s/it]


KeyError: 'mm-argfallacy-2025'

In [ ]:
split_info = loader.get_splits()  # or loader.get_splits('default')

# Access the test dataset
split_infos = loader.get_splits()

# Initialize variables to store datasets
train_dataset = val_dataset = test_dataset = None

# Iterate over the splits to assign datasets
for split_info in split_infos:
    print(dir(split_info))





['__annotations__', '__class__', '__dataclass_fields__', '__dataclass_params__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__match_args__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'test', 'train', 'val']


In [24]:
# Retrieve the data splits
split_infos = loader.get_splits()

# Initialize variables to store datasets
train_dataset = val_dataset = test_dataset = None

# Iterate over the splits to assign datasets
for split_info in split_infos:
    if hasattr(split_info, 'train'):
        train_dataset = split_info.train
    if hasattr(split_info, 'val'):
        val_dataset = split_info.val
    if hasattr(split_info, 'test'):
        test_dataset = split_info.test



In [26]:
# Check the number of samples in each split
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")

# View a sample from the training dataset
sample = train_dataset[0]
print(sample)


Training samples: 17118
Validation samples: 0
Test samples: 0
('I have a plan -- not a secret plan.', 0, None)


In [29]:
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")


Training samples: 17118
Validation samples: 0
Test samples: 0


In [35]:
# Convert to list for shuffling
train_data_list = list(train_dataset)


In [36]:
import random

# Shuffle the data
random.shuffle(train_data_list)


In [37]:
# Calculate split indices
total_samples = len(train_data_list)
train_end = int(0.8 * total_samples)
val_end = int(0.9 * total_samples)

# Split the data
train_split = train_data_list[:train_end]
val_split = train_data_list[train_end:val_end]
test_split = train_data_list[val_end:]


In [40]:
# For training data
train_samples = [sample[0] for sample in train_split]
train_labels = [sample[1] for sample in train_split]

# For validation data
val_samples = [sample[0] for sample in val_split]
val_labels = [sample[1] for sample in val_split]

# For test data
test_samples = [sample[0] for sample in test_split]
test_labels = [sample[1] for sample in test_split]


In [42]:
from mamkit.data.datasets import UnimodalDataset

# Reconstruct datasets using positional arguments
train_dataset = UnimodalDataset(train_samples, train_labels)
val_dataset = UnimodalDataset(val_samples, val_labels)
test_dataset = UnimodalDataset(test_samples, test_labels)



In [44]:
# Examine a few samples from the training dataset
for i in range(5):
    sample = test_dataset[i]
    print(f"Sample {i+1}:")
    print(f"Text: {sample[0]}")
    print(f"Label: {sample[1]}")
    print(f"Metadata: {sample[2]}\n")


Sample 1:
Text: It is a commitment that we have to meet if we're going to protect the security of Western Europe.
Label: 0
Metadata: None

Sample 2:
Text: I don't mean to be critical of uh - Senator Mondale, but uh - I was uh - very, very surprised when I read that uh - Senator Mondale made a very derogatory, very personal comment about General Brown uh - after the news story that uh - broke about General Brown.
Label: 1
Metadata: None

Sample 3:
Text: We have record numbers of new small businesses.
Label: 0
Metadata: None

Sample 4:
Text: You look at Charlotte.
Label: 0
Metadata: None

Sample 5:
Text: I was just endorsed by ICE.
Label: 0
Metadata: None



In [45]:
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")


Training samples: 13694
Validation samples: 1712
Test samples: 1712


In [47]:
import pandas as pd

# Assuming train_dataset is a list of tuples: (text, label)
data = [{'text': sample[0], 'label': sample[1]} for sample in train_dataset]

# Create a DataFrame
df = pd.DataFrame(data)

# Save to CSV
df.to_csv('train_dataset.csv', index=False)


In [1]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer

# Load model
model = RobertaForSequenceClassification.from_pretrained("./roberta_fallacy_model")
tokenizer = RobertaTokenizer.from_pretrained("./roberta_fallacy_model")

c:\Users\waral\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
# Example input
text = "America is a fair society, and it is not right that Vice President Bush pays less in taxes than the janitor who helps him."

# Tokenize the input
inputs = tokenizer(text, return_tensors="pt")

# Get model predictions
outputs = model(**inputs)
logits = outputs.logits

# Determine predicted class
predicted_class = logits.argmax().item()
print(f"Predicted class: {predicted_class}")


Predicted class: 2


In [16]:
id2label = {
    0: "Appeal to Emotion",
    1: "Appeal to Authority",
    2: "Ad Hominem",
    3: "False Cause",
    4: "Slippery Slope",
    5: "Slogans"
}


In [17]:
# Example: predicted_class is the index returned by the model
predicted_class = logits.argmax().item()
predicted_label = id2label.get(predicted_class, "Unknown")
print(f"Predicted Fallacy: {predicted_label}")


Predicted Fallacy: Appeal to Emotion


In [14]:
# Define label mappings
label2id = {
    "Equivocation": 0,
    "Faulty Generalization": 1,
    # ... add all labels
}
id2label = {v: k for k, v in label2id.items()}

# Set label mappings in model configuration
model.config.label2id = label2id
model.config.id2label = id2label

# Save the model and tokenizer
model.save_pretrained("./roberta_fallacy_model_updated")
tokenizer.save_pretrained("./roberta_fallacy_model_updated")


('./roberta_fallacy_model_updated\\tokenizer_config.json',
 './roberta_fallacy_model_updated\\special_tokens_map.json',
 './roberta_fallacy_model_updated\\vocab.json',
 './roberta_fallacy_model_updated\\merges.txt',
 './roberta_fallacy_model_updated\\added_tokens.json')